In [35]:
import selenium
import logging
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import pyautogui as pag

import time

In [36]:
driver = webdriver.Chrome('C:/Users/wynter/Desktop/DDPS/Spot.io/chromedriver')


C:\Users\wynter\AppData\Local\Temp\ipykernel_1036\2619305423.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/wynter/Desktop/DDPS/Spot.io/chromedriver')


In [37]:
driver.get('https://console.spotinst.com/spt/auth/signIn')
driver.implicitly_wait(10)
driver.maximize_window()

In [38]:
# zoom level control - minimize
pag.keyDown('ctrl')
for i in range (0, 8) :
    pag.press('-')
pag.keyUp('ctrl')

In [39]:
## login ##
# get id and pw
id = ''
pw = ''
with open('userInfo.txt', 'r') as f :
    lines = f.readlines()
    id = str(lines[0])
    pw = str(lines[1])
    
# find email, pw, btn element
get_element_email = driver.find_element(By.NAME, 'email')
get_element_pw = driver.find_element(By.NAME, 'signInPassword')
get_login_btn = driver.find_element(By.NAME, 'signIn')

# put values and click btn
get_element_email.send_keys(id)     ### need to conseal user Info ###
get_element_pw.send_keys(pw)   ### need to conseal user Info ###
get_login_btn.click()

#------------------- issue : login expired
time.sleep(6)


In [40]:
### create Elastigroup ###
# go to create Elastigroups
driver.get('https://console.spotinst.com/spt/aws/ec2/elastigroup/create/general?application=auto_scale')
time.sleep(5)

# pick a name
get_pick_name = driver.find_element(By.XPATH, '//input[@aria-label="Name"]')
get_pick_name.send_keys('test')

# Choose a region 
#-------------------- issue : sometimes region list does not appear
driver.find_element(By.NAME, 'groupRegion').click()
driver.find_element(By.XPATH, '//div[text()="Asia Pacific (Seoul)"]').click()

# press NEXT btn
get_next_btn = driver.find_element(By.XPATH, '//button[text()="Next"]')
get_next_btn.click()

time.sleep(5)


In [41]:
### Compute ###
# select VPC
driver.find_element(By.NAME, 'groupVPC').click()
driver.find_element(By.XPATH, '//md-option[@ng-value="vpc.vpcId"]').click()

# select Product - Linux/UNIX
driver.find_element(By.NAME, 'spotComputeInstanceProduct').click()
driver.find_element(By.XPATH, '//div[text()="Linux/UNIX"]').click()

# select avalability zones - all
for i in range(1, 5):
    element = driver.find_element(By.XPATH, '//*[@id="az-select"]/div[1]/div[{}]/div/group-availability-zone-name/md-input-container/md-checkbox/div[1]'.format(i))
    element.click()

In [42]:
## check Spot types
spot_types = {"General Purpose" : "general", "Compute Optimized" : "compute", "Accelerated Computing" : "accelerated", "Memory Optimized" : "memory", "Storage Optimized" : "storage"}

for type, typelist in spot_types.items() :
    # click tab
    driver.find_element(By.XPATH, '//md-tab-item[contains(text(), "{}")]'.format(type)).click()

    # get table and check
    # ------------------------- issue : too slow in exception(no checkbox) - 19m
    instance_category_tab = driver.find_element(By.XPATH, '//instance-category-tab[@instance-type-list="InstanceTypesByCategorizeCtrl.instanceTypeList.{}"]'.format(typelist))
    tbody = instance_category_tab.find_element(By.TAG_NAME, 'table').find_element(By.TAG_NAME, 'tbody')
    for tr in tbody.find_elements(By.TAG_NAME, 'tr') :
        for td in tr.find_elements(By.TAG_NAME, 'td') : ##### get attribute
            tmp_list = td.find_elements(By.TAG_NAME, 'md-checkbox')                    
            if len(tmp_list) > 0 :
                if tmp_list[0].get_attribute("aria-checked") == "false":
                    tmp_list[0].click()               
            else:
                logging.error("No Checkbox")
            
        # try :
        #     td.find_element(By.TAG_NAME, 'md-checkbox').click()
        # except :
        #     logging.error('No Checkbox')
        #     continue

ERROR:root:No Checkbox
ERROR:root:No Checkbox
ERROR:root:No Checkbox


TypeError: object of type 'NoneType' has no len()

In [ ]:
### Crawling
# find chart
chart_series_group = driver.find_element(By.CLASS_NAME, 'highcharts-series-group')
